# Combining with COSMOS 2020
Realised that downloading parts of the data previously limits what I can do with it. Therefore, redownloading the data and saving everything. Using the previously matched catalogue though, for speed.

In [1]:
import pandas as pd
import sys
import time
import numpy as np
from tqdm import tqdm
tqdm.pandas()

from astropy.io import fits

from IPython.display import clear_output

In [32]:
with fits.open('E:/temporary-cosmos-big-data/COSMOS2020_CLASSIC_R1_v2.1_p3.fits.gz') as hdul:
    data = hdul[1].data
    header = hdul[1].header

  0%|          | 0/2463 [10:27<?, ?it/s]


In [3]:
cat2015 = pd.read_csv('C:/Users/oryan/Documents/mergers-in-cosmos/data/cosmos-matched-all-real-df.csv', index_col = 0)
cat2015_red = cat2015[['SourceID', 'id_1', 'id_2']]

C:\Users\oryan\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (460) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
cat2015_red

,SourceID,id_1,id_2
0,4000705532455,590539,588102
1,4000705533312,621053,624589
2,4000705533383,617834,617637
3,4000705539529,882817,881527
4,4000705539941,893328,892731
...,...,...,...
2458,6000536184828,862874,861480
2459,6000536185304,887383,889072
2460,6000536185496,898134,898261
2461,6000536185585,904074,901845


In [5]:
ids_2020 = data['ID']
ids_2015 = data['ID_COSMOS2015']

matched_ids = pd.DataFrame(zip(ids_2020, ids_2015)).rename(columns = {0: 'ids_2020',1 : 'ids_2015'}).query('ids_2015 > -99')

In [6]:
def get_ids(sourceid, id_1_2015, id_2_2015, matched_ids):
    id_1_2020_df = matched_ids.query('ids_2015 == @id_1_2015')
    if len(id_1_2020_df) > 0 and len(id_1_2020_df) < 2:
        id_1_2020 = id_1_2020_df.ids_2020.iloc[0]
    else:
        id_1_2020 = 'failed'
            
    id_2_2020_df = matched_ids.query('ids_2015 == @id_2_2015')
    if len(id_2_2020_df) > 0 and len(id_2_2020_df) < 2:
        id_2_2020 = id_2_2020_df.ids_2020.iloc[0]
    else:
        id_2_2020 = 'failed'
    
    return {'SourceID' : sourceid, 'id_1_2020' : id_1_2020, 'id_2_2020' : id_2_2020}   

In [7]:
cat2020_ids = cat2015.assign(ids_2020 = cat2015.progress_apply(lambda row: get_ids(row.SourceID, row.id_1, row.id_2, matched_ids), axis = 1))

100%|██████████| 2463/2463 [00:42<00:00, 58.00it/s]


In [8]:
ids_2020_dict = cat2020_ids.ids_2020.to_dict()

In [9]:
ids_2020_df = pd.DataFrame.from_dict(ids_2020_dict, orient = 'index')

In [74]:
def convert_results(id_1_2020,id_2_2020, data, empty_dict):    
    ## Initialise dictionary
    export_dict = empty_dict.copy()
    
    wanted_keys = ['ez_LIR', 'ez_L2800']
    
    data_column = data[data['ID'] == id_1_2020]
    
    ## Add all values
    for i in wanted_keys:
        if len(data_column) > 0.5:
            export_dict[f'{i}_1'] = data_column[f'{i}'][0]
        else:
            export_dict[f'{i}_1'] = np.nan
        
    
    data_column = data[data['ID'] == id_2_2020]
    
    ## Assign values for secondary.
    for i in wanted_keys:
        if len(data_column) > 0.5:
            export_dict[f'{i}_2'] = data_column[f'{i}'][0]
        else:
            export_dict[f'{i}_2'] = np.nan
        
    clear_output(wait = True)
    
    return export_dict

In [66]:
init_dict = {'ez_LIR_1' : None, 'ez_L2800_1': None, 'ez_LIR_2' : None, 'ez_L2800_2': None}

In [67]:
ids_2020_dict_use = ids_2020_df.set_index('SourceID').to_dict(orient = 'index')

In [78]:
dict_results = {key : convert_results(ids_2020_dict_use[key]['id_1_2020'], ids_2020_dict_use[key]['id_2_2020'], data, init_dict) for key in tqdm(list(ids_2020_dict_use.keys()))}

100%|██████████| 2463/2463 [40:00<00:00,  1.03it/s]


In [83]:
df_eazy = pd.DataFrame.from_dict(dict_results, orient = 'index').reset_index().rename(columns = {'index' : 'SourceID'})

In [84]:
df_eazy

,SourceID,ez_LIR_1,ez_L2800_1,ez_LIR_2,ez_L2800_2
0,4000705532455,1.461746e+10,6.532970e+09,4.550346e+08,3.795326e+08
1,4000705533312,3.561132e+10,1.346792e+09,1.310930e+11,5.523424e+09
2,4000705533383,5.621026e+09,5.348111e+09,2.269957e+10,2.636891e+09
3,4000705539529,4.125722e+10,1.431745e+10,5.301575e+08,2.605845e+09
4,4000705539941,5.449620e+07,3.591865e+08,1.352380e+09,3.631612e+08
...,...,...,...,...,...
2458,6000536184828,1.478491e+11,1.198529e+10,NaN,NaN
2459,6000536185304,1.230318e+11,6.801087e+09,2.763437e+09,4.709840e+09
2460,6000536185496,3.568900e+10,7.501403e+09,5.335385e+08,7.185492e+07
2461,6000536185585,6.913122e+10,1.568826e+10,1.170207e+09,7.337763e+07


In [87]:
df_merge = ids_2020_df.merge(df_eazy, on = 'SourceID', how = 'left')
df_merge

,SourceID,id_1_2020,id_2_2020,ez_LIR_1,ez_L2800_1,ez_LIR_2,ez_L2800_2
0,4000705532455,816891,812427,1.461746e+10,6.532970e+09,4.550346e+08,3.795326e+08
1,4000705533312,873195,872551,3.561132e+10,1.346792e+09,1.310930e+11,5.523424e+09
2,4000705533383,861738,861742,5.621026e+09,5.348111e+09,2.269957e+10,2.636891e+09
3,4000705539529,1284864,1281998,4.125722e+10,1.431745e+10,5.301575e+08,2.605845e+09
4,4000705539941,1301188,1300267,5.449620e+07,3.591865e+08,1.352380e+09,3.631612e+08
...,...,...,...,...,...,...,...
2458,6000536184828,1258774,1251929,1.478491e+11,1.198529e+10,NaN,NaN
2459,6000536185304,1298440,1294806,1.230318e+11,6.801087e+09,2.763437e+09,4.709840e+09
2460,6000536185496,1308675,1308556,3.568900e+10,7.501403e+09,5.335385e+08,7.185492e+07
2461,6000536185585,1317966,1315791,6.913122e+10,1.568826e+10,1.170207e+09,7.337763e+07


In [91]:
df_export = df_merge.merge(cat2015, on = 'SourceID', how = 'left')

In [93]:
df_export.to_csv('C:/Users/oryan/Documents/mergers-in-cosmos/data/cosmos-matched-all-eazy-df.csv')